# List Full Names of all the participants in your team below:
1. Daniel Walsh
2. Faizaan Arshad
3. Jonathan Choi
4. Mayank Lara
5. Mohit Gokul Murali
6. Rishabh Kumar
7. Sagarika Suresh T
8. Shuoling Li
9. Tulika Sharma
10. 
11. 

Hello Machine Learning Engineer Sera Team, 

You have been given a **Banknote Authentication Dataset** from UCI data repository. The data features were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features (f1,f2,f3) from images.

Number of Instances: 1372 <br>
Number of Attributes: 5 (including the target variable `y`)

Attribute Information: 
   * **y:**  Forged or not (0 Forged or +1 Not Forged)
   * **f1:** variance of Wavelet Transformed image (continuous)
   * **f2:** skewness of Wavelet Transformed image (continuous)
   * **f3:** curtosis of Wavelet Transformed image (continuous)
   * **f4:** entropy of image (continuous)

There are no missing Attribute Values.

Your task is to implement a **Logistic Regression using Variational Bayes EM appraoch** for predicting given a banknote is forged or not.




## Variational Bayesian Logistic Regression

Similar to the Laplace Method for Bayesian Logistic Regression, the Variational Methods focuses on the use of Gaussian Approximation to the posterior distribution. In fact the variational approcimation to the posterior distribution leads to improved accuracy compared to the laplace method. The variational approach is optimizing a well defined objective function given by a rigorous bound on the model evidence. 



<font color="Green">USE THE VBLogisticRegression CLASS DIRECTLY.</font>

### **Question 1:** In the following code cell implement Step 5, 6 and 7. 
### **Question 2:** MAP the formulaes to the code snippets of VBLR class in Step 5.1, Step 5.2 and Step 6.1 and Step 6.2 : 
* Step 1: Import the dataset using Pandas Dataframe (Step 1 Implemented already)
* Step 2: Partition your dataset into training and testing using sklearns train_test_split library and split the features and target labels into seperate variables (Step 2 Implemented already)
* Step 3: Scale the training and testing features using sklearns min max scaling function (Step 3 Implemented already)
* Step 4: Convert Labels into numpy arrays (Step 4 Implemented already)
* Step 5: Train with Training Dataset using VBLR Solution. Below are the steps required for training VBLR model:
  * Step 5.1: Initialize Variables for Training (eps, bias (w0), parameters of approximate distribution (a,b)) 
  * Step 5.2: Run EM algorithm Iteratively to update approximate variational posterior distribution q(w, alpha): 
    * Step 5.2.1: E-Step: Update approximation of posterior distribution q(w, alpha) = q(w)*q(alpha) 
      * Update q(w) 
        * Find lambda(eps) using $\lambda (\xi) = -\frac{1}{2\xi}[\sigma(\xi) - \frac{1}{2}] = -\frac{1}{4}\frac{1 - e^{-\xi}}{\xi(1 + e^{-\xi})}$
        * Update mean and variance for approximate posterior q(w) using <br>
        $\Sigma_{N}^{-1} = E[\alpha]I + 2\Sigma_{n=1}^{N}\lambda(\xi_{n}X_{n}X_{n}^{T})$ <br>
        $\Sigma_{N}^{-1}\mu_{N} = \Sigma_{n=1}^{N} (t_{n} - \frac{1}{2})X$ <br>
        $E[\alpha] = \frac{a_{N}}{b_{N}}$
      * Update q(alpha) (a is constant, b needs to updated) <br>
        $a_{N} = a_{0} + \frac{M}{2}$ where M: Number of Features (Fixed)<br>
        $b_{N} = b_{0} + \frac{1}{2}E[w^{T}w]$ <br>
        $E[w^{T}w] = \Sigma_{N} + \mu_{N}^{T}\mu_{N}$ <br>
    * Step 5.2.2: M-Step: Update Parameters eps which controls accuracy of local variational approximation to lower bound $(\xi_{n}^{new})^{2} = X_{n}^{T}(\Sigma_{N} + \mu_{N}\mu_{N}^{T})X_{n}$
* Step 6: Test using Testing Dataset
  * Step 6.1: Use probit function equation as genesis equation:
    * $\sigma(\frac{\mu.X + bias}{\sqrt{1 + \pi\Sigma^{2}/8}})$ 
  * Step 6.2: Calculate Accuracy using Sklearns.Metrics library.

In [5]:
from scipy.special import expit, exprel
from scipy.linalg import solve_triangular

class BayesianLogisticRegression():

    def __init__(self, n_iter, tol):
        self.n_iter = n_iter
        self.tol = tol

    def fit(self, X, y):
        '''
        Fits Bayesian Logistic Regression
        Parameters
        -----------
        X: array-like of size (n_samples, n_features)
           Training data, matrix of explanatory variables

        y: array-like of size (n_samples, )
           Target values

        Returns
        -------
        self: object
           self
        '''

        # Get the number of classes
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        # Augment X with ones to include bias
        X = self._add_intercept(X)

        self.coef_, self.sigma_, self.intercept_ = [0], [0], [0]

        # make classifier for just binary classification
        coef_, sigma_ = self._fit(X, y) ## Calling VBLR Fit
        self.intercept_[0], self.coef_[0] = self._get_intercept(coef_)
        self.sigma_[0] = sigma_
        self.coef_ = np.asarray(self.coef_)
        return self

    def predict_prob(self, X):
        print("Testing Logistic Regression Model (Calculating Class Probabilities) . .")
        '''
        Predicts probabilities of targets for test set

        Parameters
        ----------
        X: array-like of size [n_samples_test,n_features]
           Matrix of explanatory variables (test set)

        Returns
        -------
        probs: numpy array of size [n_samples_test]
           Estimated probabilities of target classes
        '''
        # construct separating hyperplane
        scores = (np.dot(X,self.coef_.T) + self.intercept_).flatten()
        X = self._add_intercept(X)

        # probit approximation to predictive distribution
        sigma = self._get_sigma(X)
        ks = 1. / (1. + np.pi * sigma / 8) ** 0.5
        probs = expit(scores.T * ks).T

        return probs

# ============== VB Logistic Regression (with Jaakola Jordan bound) ==================s

def lam(eps):
    ''' Calculates lambda eps (used for Jaakola & Jordan local bound) '''
    eps = -abs(eps)
    return 0.25 * exprel(eps) / (np.exp(eps) + 1)

class VBLogisticRegression(BayesianLogisticRegression):
    '''
    Variational Bayesian Logistic Regression with local variational approximation.


    Parameters:
    -----------
    n_iter: int, optional (DEFAULT = 50 )
       Maximum number of iterations

    tol: float, optional (DEFAULT = 1e-3)
       Convergence threshold, if cange in coefficients is less than threshold
       algorithm is terminated

    fit_intercept: bool, optinal ( DEFAULT = True )
       If True uses bias term in model fitting

    a: float, optional (DEFAULT = 1e-6)
       Rate parameter for Gamma prior on precision parameter of coefficients

    b: float, optional (DEFAULT = 1e-6)
       Shape parameter for Gamma prior on precision parameter of coefficients


    Attributes
    ----------
    coef_ : array, shape = (n_features)
        Coefficients of the regression model (mean of posterior distribution)
    sigma_ : array, shape = (n_features, n_features)
        estimated covariance matrix of the weights, computed only
        for non-zero coefficients
    intercept_: array, shape = (n_features)
        intercepts

    References:
    -----------
   [1] Bishop 2006, Pattern Recognition and Machine Learning ( Chapter 10 )
   [2] Murphy 2012, Machine Learning A Probabilistic Perspective ( Chapter 21 )
    '''

    def __init__(self, n_iter=50, tol=1e-3,
                 a=1e-4, b=1e-4):
        super(VBLogisticRegression, self).__init__(n_iter, tol)
        self.a = a
        self.b = b
        # self._mask_val = 0.

    def _fit(self, X, y):
        '''
        Fits single classifier
        '''
        print("Training Logistic Regression Model with Variational Bayes EM approach . .")
        # Initialize Variables for Training (eps, bias (w0), parameters of approaximate distribution (a,b))
        eps = 1
        n_samples, n_features = X.shape
        XY = np.dot(X.T, (y - 0.5))
        w0 = np.zeros(n_features)

        # hyperparameters of q(alpha) (approximate distribution of precision
        # parameter of weights)
        a = self.a + 0.5 * n_features
        b = self.b

        for i in range(self.n_iter):
            # In the E-step we update approximation of
            # posterior distribution q(w,alpha) = q(w)*q(alpha)
            # --------- update q(w) ------------------
            l = lam(eps)
            w, Ri = self._posterior_dist(X, l, a, b, XY)
            # -------- update q(alpha) ---------------
            b = self.b + 0.5 * (np.sum(w[1:] ** 2) + np.sum(Ri[1:, :] ** 2))

            # -------- update eps  ------------
            # In the M-step we update parameter eps which controls
            # accuracy of local variational approximation to lower bound
            XMX = np.dot(X, w) ** 2
            XSX = np.sum(np.dot(X, Ri.T) ** 2, axis=1)
            eps = np.sqrt(XMX + XSX)

            # convergence
            if np.sum(abs(w - w0) > self.tol) == 0 or i == self.n_iter - 1:
                break
            w0 = w

        l = lam(eps)
        coef_, sigma_ = self._posterior_dist(X, l, a, b, XY, True)
        return coef_, sigma_

    def _add_intercept(self, X):
        '''Adds intercept to data matrix'''
        return np.hstack((np.ones([X.shape[0], 1]), X))

    def _get_intercept(self, coef):
        ''' Returns intercept and coefficients '''
        return coef[0], coef[1:]

    def _get_sigma(self, X):
        ''' Compute variance of predictive distribution'''
        return np.asarray([np.sum(np.dot(X, s) * X, axis=1) for s in self.sigma_])

    def _posterior_dist(self, X, l, a, b, XY, full_covar=False):
        '''
        Finds gaussian approximation to posterior of coefficients using
        local variational approximation of Jaakola & Jordan
        '''
        sigma_inv = 2 * np.dot(X.T * l, X)
        alpha_vec = np.ones(X.shape[1]) * float(a) / b
        alpha_vec[0] = np.finfo(np.float16).eps # Bias variable
        np.fill_diagonal(sigma_inv, np.diag(sigma_inv) + alpha_vec) # sigma inv
        R = np.linalg.cholesky(sigma_inv)
        Z = solve_triangular(R, XY, lower=True)
        mean = solve_triangular(R.T, Z, lower=False)

        Ri = solve_triangular(R, np.eye(X.shape[1]), lower=True)
        if full_covar:
            sigma = np.dot(Ri.T, Ri)
            return mean, sigma
        else:
            return mean, Ri

# Step 1 already implemented
import pandas as pd
import io
import requests

url = "https://raw.githubusercontent.com/Mihir2/BreakoutSessionDataset/master/titanic_lr.csv"
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Step 2 already implemented
import numpy as np
from sklearn.model_selection import train_test_split

output = data['y']
input = data.to_numpy()[:, 1:]
x_train, x_test, y_train, y_test = train_test_split(input, output, test_size=0.2)

# Step 3 already implemented
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_arr = scaler.fit_transform(x_train)
x_test_arr = scaler.transform(x_test)

# Step 5 already implemented
y_train_arr = y_train.to_numpy()
y_test_arr = y_test.to_numpy()

# Step 6 Train using Variational Bayesian Regression Class (Calculate model parameters)
VBLR = VBLogisticRegression()
VBLR.fit(x_train_arr,y_train_arr)


# Step 7 Test using Variational Bayesian Regression Class (Calculate class Probabilities)
y_preds = VBLR.predict_prob(x_test_arr)
y_preds[y_preds>=0.5]=1
y_preds[y_preds<0.5]=0

#6.2
# Step 8 Calculate Accuracy using Sklearns library
from sklearn.metrics import accuracy_score
accuracy_score(y_test_arr.flatten(),y_preds.flatten())

Training Logistic Regression Model with Variational Bayes EM approach . .
Testing Logistic Regression Model (Calculating Class Probabilities) . .


0.7132867132867133

In [13]:
import pandas as pd
import io
import requests
url = "https://raw.githubusercontent.com/Mihir2/BreakoutSessionDataset/master/data_banknote_authentication_lr.csv"
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')))

import numpy as np
from sklearn.model_selection import train_test_split
output = data['y']
input = data.to_numpy()[:, 1:]
x_train, x_test, y_train, y_test = train_test_split(input, output, test_size=0.2)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_arr = scaler.fit_transform(x_train)
x_test_arr = scaler.transform(x_test)
y_train_arr = y_train.to_numpy()
y_test_arr = y_test.to_numpy()

tol = 0.001
a = 0.0001
eps = 1
n_samples, n_features = x_train_arr.shape
w0 = np.zeros(n_features + 1)
coef, sigma, intercept = [0], [0], [0]
n_iter = 100
a = a + 0.5 * n_features
b = 0.0001
x_train_arr = np.hstack((np.ones([x_train_arr.shape[0], 1]), x_train_arr))

from scipy.special import expit, exprel
from scipy.linalg import solve_triangular
for i in range(n_iter):
    eps = -abs(eps)
    l = 0.25 * exprel(eps) / (np.exp(eps) + 1)

    sigma_inv = 2 * np.dot(x_train_arr  .T * l, x_train_arr)
    alpha_vec = np.ones(x_train_arr.shape[1]) * float(a) / b
    alpha_vec[0] = np.finfo(np.float16).eps
    np.fill_diagonal(sigma_inv, np.diag(sigma_inv) + alpha_vec)
    R = np.linalg.cholesky(sigma_inv)
    Z = solve_triangular(R, np.dot(x_train_arr.T, (y_train_arr- 0.5)), lower=True)
    mean_ = solve_triangular(R.T, Z, lower=False)
    sigma_ = solve_triangular(R, np.eye(x_train_arr.shape[1]), lower=True)

    b = b + 0.5 * (np.sum(mean_[1:] ** 2) + np.sum(sigma_[1:, :] ** 2))

    XMX = np.dot(x_train_arr, mean_) ** 2
    XSX = np.sum(np.dot(x_train_arr, sigma_.T) ** 2, axis=1)
    eps = np.sqrt(XMX + XSX)

    if np.sum(abs(mean_ - w0) > tol) == 0 or i == n_iter - 1:
      break
    w0 = mean_

intercept[0] = mean_[0]
sigma[0] = sigma_
mean = mean_[1:]
scores = (np.dot(x_test_arr,mean.T) + intercept).flatten()
x_test_arr = np.hstack((np.ones([x_test_arr.shape[0], 1]), x_test_arr))
si = np.asarray([np.sum(np.dot(x_test_arr, s) *x_test_arr, axis=1) for s in sigma])
ks = ks = 1. / (1. + np.pi * si / 8) ** 0.5
y_pred = expit(scores.T * ks).T
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0

from sklearn.metrics import accuracy_score
accuracy_score(y_test_arr.flatten(),y_pred.flatten())

0.9890909090909091